Welcome to analysis of **"Covid-19" Impact on Digital Learning**.

Objectives :-
Understanding impact of Covid on the digital platforms in student's engagement and more.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/learnplatform-covid19-impact-on-digital-learning/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

We notice, there are **3 files** in root and a folder named **engagement_data** with many CSVs. 

Viewing the files

In [ ]:
products = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
districts = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")

Products file

In [ ]:
products.head()

In [ ]:
products.info()

In [ ]:
products.describe(include='all')

In [ ]:
products.groupby('Sector(s)').size()

Districts file

In [ ]:
districts.head()

In [ ]:
districts.describe(include='all')

Now sample file inside the folder **engagement_data**.

In [ ]:
d6345 = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/engagement_data/6345.csv")


In [ ]:
d6345.head()

In [ ]:
d6345.describe(include='all')

We will merge all data files to one file.

In [ ]:
appended_file = []
for dirname, _, filenames in os.walk('../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'):
    for filename in filenames:
        temp_file = pd.read_csv(f'../input/learnplatform-covid19-impact-on-digital-learning/engagement_data/{filename}')
        temp_file["district_idd"] = filename
        appended_file.append(temp_file)

In [ ]:
engagement = pd.concat(appended_file)
engagement = engagement.reset_index(drop=True)

In [ ]:
engagement.describe(include='all')

In [ ]:
engagement.dtypes

In [ ]:
a = pd.DataFrame(engagement.groupby('district_idd')['time'].nunique())

In [ ]:
a['is_366'] = np.where(a['time']== 366,1,0)

In [ ]:
a.groupby('is_366').size()

This shows that there are 61 districts with less than 366 days of information. Excluding those, since for arriving at relation between engagement and other paramters, the engagement should be of all days.


In [ ]:
appended_file = []
for dirname, _, filenames in os.walk('../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'):
    for filename in filenames:
        temp_file = pd.read_csv(f'../input/learnplatform-covid19-impact-on-digital-learning/engagement_data/{filename}')
        temp_file["district_idd"] = filename
        if temp_file.time.nunique() == 366:            
            appended_file.append(temp_file)
       # print(os.path.join(dirname, filename))

In [ ]:
engagement = pd.concat(appended_file)
engagement = engagement.reset_index(drop=True)
engagement['Date']= pd.to_datetime(engagement['time'])
engagement

In [ ]:
engagement.groupby(['district_idd'])['time'].nunique().describe()

This confirms that all district now have 366 entries. The data files are merged in one file names engagement.
Now we have 3 types of files.
1. Engagement
2. Products
3. Districts

In [ ]:
engagement.describe(include='all')

In [ ]:
products.describe(include='all')

In [ ]:
districts.describe(include='all')

Merging all 3 to 1 file and keeping only important fields.

In [ ]:
base_file = engagement.merge(products, left_on= 'lp_id', right_on='LP ID', how = 'left')

In [ ]:
base_file['district_id'] = base_file['district_idd'].str.slice(0, 4).astype(int)

In [ ]:
base_file2 = base_file.merge(districts, left_on= 'district_id', right_on='district_id', how = 'left')

In [ ]:
base_file2.describe(include ='all')

In [ ]:
base_file3 = base_file2[['lp_id','pct_access','engagement_index','Date','Product Name','Provider/Company Name','Sector(s)','Primary Essential Function','district_id','state','locale','pct_black/hispanic','pct_free/reduced','county_connections_ratio','pp_total_raw']]

In [ ]:
base_file2.columns

In [ ]:
base_file3.columns

In [ ]:
base_file3.describe(include='all')

Summarising and studying how different parameters interact. We should make charts and study how different value of parameters behave against others.
Categorical variables to be studied
1. Week/Day/month
2. Company
3. Product Names
4. Sectors
5. Primary Function
6. District Id
7. State
8. Local
9. Percentage Black/Hispanic [Convert this into categorical bins]

Continous Variables against which above categorical variables can be compared
1. Pct_access
2. engagement index
3. Percentage Black/Hispanic
4. pct_free/reduced	
5. pp_total_raw

In [ ]:
summary_sector = base_file3.groupby(['Sector(s)']).agg(engagement_index_sum=("engagement_index", pd.Series.sum), 
                                             engagement_index_mean = ("engagement_index", pd.Series.mean),
                                             engagement_index_mean2 = ("engagement_index", np.mean),
                                             unique_date=("Date", pd.Series.nunique), 
                                             count_date=("Date", pd.Series.count), 
                                            ).reset_index()

In [ ]:
summary

In [ ]:
summary = base_file3.groupby(['Sector(s)']).agg(engagement_index_sum=("engagement_index", pd.Series.sum), 
                                             engagement_index_mean = ("engagement_index", pd.Series.mean),
                                             engagement_index_mean2 = ("engagement_index", np.mean),
                                             unique_date=("Date", pd.Series.nunique), 
                                             count_date=("Date", pd.Series.count), 
                                            ).reset_index()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
figz = sns.barplot(summary['Sector(s)'], summary['engagement_index_mean'])
set(figz,'Position',[100, 100, 500, 500])
figz.Position = [10, 10, 550, 400]; 
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.barplot(summary['Sector(s)'], summary['engagement_index_mean'])
plt.gcf().set_size_inches(30,10)
plt.show()

In [ ]:
summary2 = base_file3.groupby(['Primary Essential Function']).agg(engagement_index_sum=("engagement_index", pd.Series.sum), 
                                             engagement_index_mean = ("engagement_index", pd.Series.mean),
                                             engagement_index_mean2 = ("engagement_index", np.mean),
                                             unique_date=("Date", pd.Series.nunique), 
                                             count_date=("Date", pd.Series.count), 
                                            ).reset_index()

In [ ]:
summary2

We should split this 1st column

In [ ]:
summary2['Primary_EF'] = summary2['Primary Essential Function'].str.rsplit("-",n=0, expand=True)

In [ ]:
summary2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.barplot(summary2['Primary_EF'], summary2['engagement_index_mean'])
plt.gcf().set_size_inches(30,10)
plt.show()

In [ ]:
new_df

In [ ]:
summary2b = summary2

In [ ]:
summary2b['new'] = data

In [ ]:
summary2b